## 0. Introduction:

ClinicalTrials (https://clinicaltrials.gov) is a registry and results database of publicly and privately supported research studies conducted in the United States and around the world. Sponsors or investigators of certain clinical trials are required by U.S. law to register their trials on and submit summary results to ClinicalTrials.gov. Users can freely find clinical trials of interest by using the Advanced Search. In this notebook, we will go through step by step to:

- Access the clinicaltrial database by webscraping **nct_ids** which match the search criteria. 
    - **nct_ids** (NCT ID) is a unique identification code given to each clinical study registered on ClinicalTrials.gov.
- Extract datasets from **AACT** database by using the **nct_ids** from above and generate the final output.

In the final output, we will have a database for people responsible for studies and show what clinical trials are associated with those people, corresponding **nct_ids**, number of trials, etc. 

## 1. Load Necessary Libraries

You will need to install below libraries first in order to run the codes below to load them. One way to install these modules is to go to Anaconda Prompt type **conda install library_name_goes_here** to try to install them from conda. If cannot found in conda, type **pip install library_name_goes_here** to install them from pip.

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import itertools 
from cytoolz import *
import urllib
import xmltodict
from collections import Counter
import zipfile
import os
import math
from tqdm import *
# from tqdm import tnrange, tqdm_notebook
# import dataset

## 2. Advanced Search 

Use ClinicalTrials.com Advanced Search to enter the customized search terms: 

1. Go to https://clinicaltrials.gov/ct2/search/advanced?cond=&term=&state1=&cntry1=

<img src='Clinicaltrials.png' width="800" height="800">

2. To use the Advanced Search form, enter search terms in **one** or **more** fields and then click on Search.
3. A list of search results will be displayed. 
4. See search field names and the corresponding tag names in HTTP below. 

In [2]:
# In the following dictionary, the keys represent field names on the webpage, the values represent HTTP query terms
query_key_dic = {
'Condition/Disease': 'cond',
'Other Terms': 'term',
'Study Type': 'type', # see select_options.json for how to send the correct query value
'Study Results': 'rslt', # see select_options.json for how to send the correct query value
'Status': 'recrs', # see checkbox_options.json for how to send the correct query value
'Age': 'age_v',
'Age Group': 'age', # see checkbox_options.json for how to send the correct query value
'Sex': 'gndr', # see select_options.json for how to send the correct query value
'Accepts Health Volunteers': 'hlth', # see checkbox_options.json for how to send the correct query value
'Intervention/Treatment': 'intr',
'Title Acronym/Titles': 'titles',
'Outcome Measures': 'outc',
'Sponsor/Collaborators': 'spons',
'Sponsor(Lead)': 'lead',
'Study IDs': 'id',
'Country': 'cntry1', # see select_options.json for how to send the correct query value
'State': 'state1', # see select_options.json for how to send the correct query value
'Country 2': 'cntry2', # see select_options.json for how to send the correct query value
'State 2': 'state2', # see select_options.json for how to send the correct query value
'Country 3': 'cntry3', # see select_options.json for how to send the correct query value
'State 3': 'state3', # see select_options.json for how to send the correct query value
'Location Terms': 'locn',
'Phase': 'phase', # see checkbox_options.json for how to send the correct query value
'Funder Type': 'fund', # see checkbox_options.json for how to send the correct query value
'First Received From': 'rcv_s',
'First Received To': 'rcv_e',
'Last Updated From': 'lup_s',
'Last Updated To': 'lup_e'    
}

## 3. Set up Search Terms 

In [3]:
# We will use the following search terms to find clinical trial studies.
search_term = pd.read_excel("Search Terms.xlsx",sheet_name ='CT', skiprows = 1, usecols  = 'D:H', skipfooter=35)
search_term

,ID,StudyType,Condition/Disease,Intervention/Treatment,Other Terms
0,1,Interventional Studies,diabetic kidney disease OR diabetic nephropathy,None,None
1,2,Interventional Studies,None,atrasentan,None
2,3,Interventional Studies,None,empagliflozin,None
3,4,Interventional Studies,None,canagliflozin,None
4,5,Interventional Studies,None,spironolactone,None
5,6,Interventional Studies,None,eplerenone,None
6,7,Interventional Studies,None,finerenone,None
7,8,Interventional Studies,diabetes OR diabetic kidney disease OR diabeti...,None,microalbuminuria
8,9,Interventional Studies,diabetes OR diabetic kidney disease OR diabeti...,None,macroalbuminuria
9,10,Interventional Studies,diabetes OR diabetic kidney disease OR diabeti...,None,hyperkalemia


## 4. Access ClinicalTrials Data
We will webscrape clinical trial data from clinicaltrials.gov and fetch **nct_ids** which match the search criteria.
**nct_ids** (NCT ID) is a unique identification code given to each clinical study registered on ClinicalTrials.gov. **nct_ids** is the primary key in the clinicaltrials data.

In [4]:
# Set up function to query data from clinicaltrial.com
base_url = 'https://clinicaltrials.gov/ct2/download_studies'
all_records = []
def get_records(actual_query, ID):
    for query in [actual_query]:
        url = '?'.join([base_url,urllib.parse.urlencode(query)])
        with open('data.zip', 'wb') as file:
            res = requests.get(url, stream = True)
            file.write(res.content)
            res.close()
        zip_ref = zipfile.ZipFile('data.zip', 'r')
        zip_ref.extractall('.')
        zip_ref.close() 
        os.remove('data.zip')    
        files = [x for x in os.listdir('.') if os.path.splitext(x)[1] == '.xml']
        for name in files:
            with open(name, 'r', encoding = 'latin1') as file:
                new_record = xmltodict.parse(file.read())
                new_record['Match_search_ID']=ID
                all_records.append(new_record)
            os.remove(name)

In [5]:
# Scrape data from clinicaltrial.com by using specified criteria
Time_frame={'First Received From': '01/01/2008', 'First Received To': '12/31/2018'}

for index, row in tqdm(search_term.iterrows(), total=search_term.shape[0]):
    query={'Study Type':row['StudyType'], 'Condition/Disease':row['Condition/Disease'], 'Intervention/Treatment':row['Intervention/Treatment'], 'Other Terms':row['Other Terms']}
    # update query with additional criteria
    query.update(Time_frame)
    query={k: v for k, v in query.items() if v!='None'}
    # finalized query that we will apply for search
    actual_query = {query_key_dic[key]: value for key, value in query.items()}
    # query data from clinicaltrial.com
    get_records(actual_query, row['ID'])

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:47<00:00,  4.35s/it]


In [6]:
all_records[7]

OrderedDict([('clinical_study',
              OrderedDict([('@rank', '185'),
                           ('required_header',
                            OrderedDict([('download_date',
                                          'ClinicalTrials.gov processed this data on January 21, 2022'),
                                         ('link_text',
                                          'Link to the current ClinicalTrials.gov record.'),
                                         ('url',
                                          'https://clinicaltrials.gov/show/NCT00713011')])),
                           ('id_info',
                            OrderedDict([('org_study_id', '12716'),
                                         ('nct_id', 'NCT00713011')])),
                           ('brief_title',
                            'Adalat XL vs Diltiazem on Proteinuria and Blood Pressure in Hypertensive Diabetic Patients'),
                           ('acronym', 'CARDINAL'),
                          

In [7]:
len(all_records)

1309

In [8]:
match_id = pd.DataFrame([record['Match_search_ID'] for record in all_records])

In [9]:
# check number of clinicaltrials by search ID
match_id[0].value_counts().reset_index().sort_values('index')

,index,0
1,1,270
10,2,7
2,3,182
5,4,121
3,5,157
6,6,81
9,7,11
4,8,126
8,9,39
7,10,41


In [10]:
# Save the datainto json file
with open('searched_records_01222022.json', 'w') as file:
    json.dump(all_records, file)

In [11]:
# Read the data from json file
with open('searched_records_01222022.json', 'r') as file:
    all_records = json.load(file)

In [12]:
all_records[7]

{'clinical_study': {'@rank': '185',
  'required_header': {'download_date': 'ClinicalTrials.gov processed this data on January 21, 2022',
   'link_text': 'Link to the current ClinicalTrials.gov record.',
   'url': 'https://clinicaltrials.gov/show/NCT00713011'},
  'id_info': {'org_study_id': '12716', 'nct_id': 'NCT00713011'},
  'brief_title': 'Adalat XL vs Diltiazem on Proteinuria and Blood Pressure in Hypertensive Diabetic Patients',
  'acronym': 'CARDINAL',
  'official_title': 'Randomized Open-label 2-arm Parallel Design Comparator Study of the Effect of AdalatÂ® XLÂ® Compared to Diltiazem on Proteinuria and Blood Pressure in Patients With Diabetes and Mild to Moderate Hypertension When Used as an Add on to AvalideÂ®',
  'sponsors': {'lead_sponsor': {'agency': 'Bayer',
    'agency_class': 'Industry'}},
  'source': 'Bayer',
  'oversight_info': {'has_dmc': 'No'},
  'brief_summary': {'textblock': 'The study consists of a 12 week run-in period when all subjects are stabilized on a single\r

In [13]:
# check number of unique nct_ids
nct_ids = [record['clinical_study']['id_info']['nct_id'] for record in all_records]
nct_ids = list(set(nct_ids))
len(nct_ids)

1090

In [15]:
# pd.DataFrame([record['clinical_study']['id_info']['nct_id'] for record in all_records]).to_csv('nct_ids.csv')

## 5. Access ClinicalTrials.gov via AACT database 

- **What is AACT**: AACT is a publicly available relational database that contains all information (protocol and result data elements) about every study registered in ClinicalTrials.gov. Content is downloaded from ClinicalTrials.gov daily and loaded into AACT.

- **Data Files in AACT**: Aggregated content of ClinicalTrials.gov are generated as different datasets. See some examples in below:
    - *facility_contacts*: Contact information for people responsible for the study at each facility.
    - *facility_investigators*: Names of the investigators at each study facility. 
    - *overall_officials*: People responsible for the overall scientific leadership of the protocol including the principal investigator.
    - *facilities*: Name, address and recruiting status of the facilities participating in the study.
    - *responsible_parties*:responsible_parties
    - *central_contacts*: Contact info for people (primary & backup) who can answer questions concerning enrollment at any location of the study.
    - *studies*: Basic info about study, including study title, date study registered with ClinicalTrials.gov, date results first posted to ClinicalTrials.gov, dates for study start and completion, phase of study, enrollment status, planned or actual enrollment, number of study arms/groups, etc.

- **Download AACT**: A static copy of the AACT database is created on the first of each month. The most recent and archived copies are available for download. These can be used to create a local copy of a particular instance of the database. 

- **What we do**: We will download the pipe delimited data in Monthly Archives from https://aact.ctti-clinicaltrials.org/download.
    - Import the data into python
    - Extract data based on **nct_ids** that we get in the previous step
    - Further filter **nct_id** for countries of interest
    - combine all the useful information from different datasets to get the final database.

<img src='AACT database.png' width="600" height="600">


In [14]:
# Download AACT data and save it locally
# We download the pipe-delimited data in Monthly Archives from https://aact.ctti-clinicaltrials.org/pipe_files
data_month='20220101'

In [15]:
# Read AACT database into Python
aact = zipfile.ZipFile(data_month+'_pipe-delimited-export.zip')
# aact.namelist()
facility_contacts = pd.read_csv(aact.open('facility_contacts.txt'),sep='|',low_memory=False)
facility_investigators=pd.read_csv(aact.open('facility_investigators.txt'),sep='|',low_memory=False)
overall_officials=pd.read_csv(aact.open('overall_officials.txt'),sep='|',low_memory=False)
facilities=pd.read_csv(aact.open('facilities.txt'),sep='|',low_memory=False)
responsible_parties=pd.read_csv(aact.open('responsible_parties.txt'),sep='|',low_memory=False)
central_contacts=pd.read_csv(aact.open('central_contacts.txt'),sep='|',low_memory=False)
studies=pd.read_csv(aact.open('studies.txt'),sep='|',low_memory=False)

In [16]:
db={'facility_contacts':facility_contacts,
    'facility_investigators':facility_investigators, 
    'overall_officials':overall_officials, 
    'facilities':facilities, 
    'responsible_parties':responsible_parties, 
    'central_contacts':central_contacts, 
    'studies':studies}

In [17]:
# Only keep clinical trials with nct_ids that we get in the previous step 
tables_to_query = ['facility_contacts', 'facility_investigators', 'overall_officials', 'facilities', 'responsible_parties', 'central_contacts','studies']
all_useful = {key: db[key].loc[db[key]['nct_id'].isin(nct_ids)] for key in tables_to_query}

In [18]:
# Dedup the data by nct_id, country
all_useful['facilities'] = all_useful['facilities'].drop_duplicates(subset=['nct_id', 'country'])
all_useful['facilities'].country.unique()

array(['United States', 'Austria', 'Japan', 'Argentina', 'Bulgaria',
       'Canada', 'Czech Republic', 'India', 'Malaysia', 'Mexico',
       'Poland', 'Puerto Rico', 'Romania', 'Russian Federation',
       'United Kingdom', 'China', 'Korea, Republic of', 'Saudi Arabia',
       'Sweden', 'Peru', 'Brazil', 'Finland', 'France', 'Italy',
       'Switzerland', 'Nepal', 'Egypt', 'Czechia', 'Slovenia',
       'Australia', 'Indonesia', 'Spain', 'Israel', 'Germany',
       'Hong Kong', 'Denmark', 'Taiwan', 'Greece', 'Estonia', 'Hungary',
       'Latvia', 'Norway', 'Slovakia', 'Ukraine', 'Netherlands',
       'Singapore', 'Belgium', 'Colombia', 'Thailand', 'Turkey', 'Iraq',
       'Croatia', 'Lithuania', 'Iceland', 'Qatar', 'Portugal', 'Georgia',
       'South Africa', 'Chile', 'Pakistan', 'Philippines', 'Ecuador',
       'New Zealand', 'Uruguay', 'Iran, Islamic Republic of', 'Ireland',
       'Serbia', 'Cayman Islands', 'Kuwait', 'Lebanon',
       'United Arab Emirates', 'Belarus', 'Bosnia and

In [19]:
len(all_useful['facilities'].nct_id.unique()), all_useful['facilities'].shape

(984, (2394, 8))

In [20]:
# Define sub_nct_ids: Further filter nct_id by only keeping Countries = UK, DE, JP, US, FR
interested_countries = ['United Kingdom','Germany','Japan','United States','France']
facilities = all_useful['facilities'].loc[all_useful['facilities']['country'].isin(interested_countries)]

In [21]:
sub_nct_ids = facilities.nct_id.unique()

In [22]:
len(sub_nct_ids), facilities.shape

(555, (700, 8))

In [23]:
# Define all_useful: Further filter AACT database for sub_nct_ids
all_useful = {key: db[key].loc[db[key]['nct_id'].isin(sub_nct_ids)] for key in tables_to_query}

In [24]:
# Combine information from different datasets to get the final output - people table
people = (
    all_useful['facility_contacts']
        .assign(role = lambda df: df['contact_type'].str.cat(np.repeat('contact', df.shape[0]), sep = ' ').str.title())
        .drop(['id', 'contact_type'], axis=1)
).merge(
    all_useful['facility_investigators'].drop('id', axis=1), how = 'outer', on = ['nct_id', 'role', 'name']
).merge(
    all_useful['overall_officials'].drop('id', axis=1), how = 'outer', on = ['nct_id', 'role', 'name']
).merge(
    all_useful['responsible_parties']
    .assign(role = lambda df: df['responsible_party_type'].fillna('').str.cat(np.repeat('responsible party', df.shape[0]), sep = ' ').str.title())
    .drop(['id', 'responsible_party_type', 'title'], axis=1), how = 'outer', on = ['nct_id', 'role', 'name']
).merge(
    all_useful['central_contacts']
    .assign(role = lambda df: df['contact_type'].str.cat(np.repeat('contact', df.shape[0]), sep = ' ').str.title())
    .drop(['id', 'contact_type'], axis=1), how = 'outer', on = ['nct_id', 'role', 'name']
)

people = (
    people.assign(facility_id = lambda df: df.apply(lambda row: {x for x in set([row['facility_id_x'], row['facility_id_y']]) if pd.notnull(x)}, axis=1), 
        email = lambda df: df.apply(lambda row: {x for x in set([row['email_x'], row['email_y']]) if pd.notnull(x)}, axis=1),
        affiliation = lambda df: df.apply(lambda row: {x for x in set([row['affiliation_x'], row['affiliation_y']]) if pd.notnull(x)}, axis=1),
        phone = lambda df: df.apply(lambda row: {x for x in set([row['phone_x'], row['phone_y']]) if pd.notnull(x)}, axis=1)
  ).drop(['facility_id_x', 'facility_id_y', 'email_x', 'email_y', 'affiliation_x', 'affiliation_y', 'phone_x', 'phone_y'], axis=1)
)

In [25]:
people.shape, people.head()

((1772, 8),
         nct_id                                               name  \
 0  NCT01422759                                    Melissa Gilrain   
 1  NCT01712620  For more information at the NIH Clinical Cente...   
 2  NCT01774656                             Terry Blanton, BSN, RN   
 3  NCT01774656                                Stacy Flickbohm, RN   
 4  NCT01774656                                      Natalie Kamtz   
 
               role organization   facility_id                      email  \
 0  Primary Contact          NaN  {10305421.0}        {pcos@virginia.edu}   
 1  Primary Contact          NaN  {10378869.0}          {prpl@cc.nih.gov}   
 2  Primary Contact          NaN  {10393230.0}  {mtblan02@louisville.edu}   
 3  Primary Contact          NaN  {10393231.0}  {stacy.fickbohm@unmc.edu}   
 4   Backup Contact          NaN  {10393231.0}   {natalie.kamtz@unmc.edu}   
 
   affiliation           phone  
 0          {}  {434-243-6911}  
 1          {}  {800-411-1222}  
 2 

In [26]:
# Remove record with null name
people = people.loc[~people['name'].isnull()]
people.shape, people.tail()

((1338, 8),
            nct_id                                   name             role  \
 1767  NCT02768948                        Mélanie Bonnard   Backup Contact   
 1768  NCT02627690  Marie-Christine MC VANTYGHEM, MD, PhD  Primary Contact   
 1769  NCT02585622                   Giuseppe Remuzzi, MD  Primary Contact   
 1770  NCT02490904                   Frédéric JAISSER, MD  Primary Contact   
 1771  NCT03683069                         Isabella Stone   Backup Contact   
 
      organization facility_id                                      email  \
 1767          NaN          {}                    {bonnard.m@chu-nice.fr}   
 1768          NaN          {}  {marie-christine.vantyghem@chru-lille.fr}   
 1769          NaN          {}           {giuseppe.remuzzi@marionegri.it}   
 1770          NaN          {}               {frederic.jaisser@inserm.fr}   
 1771          NaN          {}                   {istone@bwh.harvard.edu}   
 
      affiliation             phone  
 1767          {

In [27]:
# Add in country and search criteria id
people_w_country = people.merge(
    facilities[['nct_id','country']].drop_duplicates(), how='left', on= ['nct_id'])
nct_id_match = pd.DataFrame([[record['clinical_study']['id_info']['nct_id'], record['Match_search_ID']] for record in all_records], columns=['nct_id','match_search_id'])
nct_id_match = nct_id_match.groupby(['nct_id']).match_search_id.unique().reset_index()

In [28]:
nct_id_match.head()

,nct_id,match_search_id
0,NCT00586261,[11]
1,NCT00594152,[1]
2,NCT00604006,[5]
3,NCT00608465,[6]
4,NCT00625820,[11]


In [29]:
# Add in brief title, match_search_id, etc
people = people.merge(
    people_w_country.groupby(['nct_id', 'role', 'name']).country.unique().reset_index(), how='left', on=['nct_id', 'role', 'name']
).merge(
    all_useful['studies'][['nct_id','brief_title']], how='left', on=['nct_id']
).merge(
    nct_id_match, how='left', on=['nct_id']
)

In [30]:
people.shape, people.tail()

((1338, 11),
            nct_id                                   name             role  \
 1333  NCT02768948                        Mélanie Bonnard   Backup Contact   
 1334  NCT02627690  Marie-Christine MC VANTYGHEM, MD, PhD  Primary Contact   
 1335  NCT02585622                   Giuseppe Remuzzi, MD  Primary Contact   
 1336  NCT02490904                   Frédéric JAISSER, MD  Primary Contact   
 1337  NCT03683069                         Isabella Stone   Backup Contact   
 
      organization facility_id                                      email  \
 1333          NaN          {}                    {bonnard.m@chu-nice.fr}   
 1334          NaN          {}  {marie-christine.vantyghem@chru-lille.fr}   
 1335          NaN          {}           {giuseppe.remuzzi@marionegri.it}   
 1336          NaN          {}               {frederic.jaisser@inserm.fr}   
 1337          NaN          {}                   {istone@bwh.harvard.edu}   
 
      affiliation             phone           country

In [31]:
# Generate the final output - nct_id_lookup table
nct_id_match.match_search_id = nct_id_match.match_search_id.apply(lambda x:{i for i in x})
nct_id_lookup = people[["nct_id","brief_title"]].copy().drop_duplicates().merge(nct_id_match, how='left', on=['nct_id'])

In [32]:
# Investigate people
# assume the honorifics are separated by commas
people['simple_name'] = people['name'].str.split(',', expand= False).str.get(0).str.replace('[^\w ]', '', regex=True).str.replace(' +', ' ', regex=True).str.title().str.strip().str.split(' ', expand = False).map(
       lambda row: ' '.join([row[0], row[-1]] if row else ''))

people_in_trials = people[['nct_id', 'role', 'simple_name', 'facility_id']]

people_agg = (
people[['nct_id','name', 'email', 'phone', 'affiliation', 'simple_name', 'organization','country','brief_title','match_search_id']]
.groupby('simple_name')
.apply(lambda df: pd.Series({
    'name': set(df['name'].dropna()),
    'email': {x for x in itertoolz.concat(df['email'].dropna()) if pd.notnull(x)},
    'affiliation': {x for x in itertoolz.concat(df['affiliation'].dropna()) if pd.notnull(x)},
    'organization': set(df['organization'].dropna()),
    'country': {x for x in itertoolz.concat(df['country'].dropna()) if pd.notnull(x)},
    'Titles': set(df['brief_title']),
    'nct_ids':set(df['nct_id']),
    'match_search_id':{x for x in itertoolz.concat(df['match_search_id'].dropna()) if pd.notnull(x)},
}))
.reset_index()
.merge(
    people_in_trials
    .groupby('simple_name')
    .apply(lambda df: pd.Series({'num_trials': df['nct_id'].drop_duplicates().shape[0]}))
    .reset_index(),
    how = 'outer', on = 'simple_name'
)
.assign(num_trials = lambda df: df['num_trials'].fillna(0))
.sort_values('num_trials',ascending=False)
)

people_agg = people_agg.replace(set(), np.nan)

In [33]:
people_agg.head()

,simple_name,name,email,affiliation,organization,country,Titles,nct_ids,match_search_id,num_trials
83,Boehringer Ingelheim,{Boehringer Ingelheim},{clintriage.rdg@boehringer-ingelheim.com},{Boehringer Ingelheim},{},"{Japan, United Kingdom, Germany, United States...",{Efficacy and Safety of Empagliflozin (BI 1077...,"{NCT01634100, NCT02453555, NCT02580591, NCT028...","{11, 9, 3, 1}",71
341,Janssen Development,"{Janssen Research & Development, LLC L.L.C Cli...",{},"{Janssen Research & Development, LLC}",{},"{Japan, United Kingdom, Germany, United States...",{A Study of Effects of Canagliflozin as Add-on...,"{NCT01137812, NCT02039245, NCT02077803, NCT010...","{1, 3, 4, 8, 9, 11}",38
386,Johnson Development,{Johnson & Johnson Pharmaceutical Research and...,{},{Johnson & Johnson Pharmaceutical Research & D...,{},"{Germany, United Kingdom, United States}","{Study to Assess the Pharmacokinetics, Pharmac...","{NCT01454622, NCT01173549, NCT01177163, NCT009...",{4},17
673,Pfizer Center,{Pfizer CT.gov Call Center},{},{Pfizer},{},"{Japan, United Kingdom, Germany, United States...",{A Study To Evaluate The Safety And Tolerabili...,"{NCT01712061, NCT01176968, NCT03342690, NCT009...","{1, 4, 5, 6, 9, 10, 11}",11
72,Bayer Director,{Bayer Study Director},{},{Bayer},{},"{Japan, United Kingdom, Germany, United States...",{Phase IIb Safety and Efficacy Study of BAY94-...,"{NCT01687920, NCT01473108, NCT01968668, NCT025...","{1, 5, 6, 7}",10


In [34]:
# Output into one single excel
writer = pd.ExcelWriter('Clinial Trial Search.xlsx')
people_agg.to_excel(writer,sheet_name='People',index=False)
nct_id_lookup.to_excel(writer,sheet_name='nct_id_lookup',index=False)
search_term.to_excel(writer,sheet_name='search_terms',index=False)
writer.save()